In [64]:
import json
import pandas as pd

In [65]:
gbsum = pd.read_csv('allCRISPRAnnotationData/gbffsumarry.csv',delimiter='~')
ncbi = pd.DataFrame(json.load(open('allCRISPRAnnotationData/ncbiAnnotations_02_11_18SR.json')))
cone = pd.DataFrame(json.load(open('allCRISPRAnnotationData/CRISPRone_files/mp_CRISPRoneAnnotations_01_11_18SR.json')))

In [66]:
ncbi = ncbi.fillna('None') #fill NaN floats with string 'None'
ncbi['GCF'] = ncbi['RefSeq assembly accession'].apply(lambda x: x.strip(' (latest)'))

In [67]:
ncbi.iloc[0]

Assembly anomaly                                                                               None
Assembly level                                                                      Complete Genome
Assembly method                                                                                None
BioProject                                                                                 PRJNA236
BioSample                                                                              SAMEA1705914
Date                                                                                     2003/05/06
Excluded from RefSeq                                                                           None
Expected final version                                                                         None
GCA                                                                      GCA_000195995.1_ASM19599v1
GenBank assembly accession                                                 GCA_000195995.1 (latest)


In [68]:
gbsum.iloc[0]

Filepath                /home/sid/thesis_SidReed/bacterialGBFFs/rawdat...
GCF                                                       GCF_000005825.2
Accession Number                                                NC_013791
Organism                                        Bacillus pseudofirmus OF4
Genus                                                            Bacillus
Taxonomy                Bacteria;Firmicutes;Bacilli;Bacillales;Bacilla...
IsComplete                                                   full length.
NCBI_CRISPR_Arrays                                                      0
Genes (total)                                                        4263
CDS (total)                                                          4159
Genes (coding)                                                       4104
CDS (coding)                                                         4104
Genes (RNA)                                                           104
tRNAs                                 

In [75]:
cn = pd.merge(cone, ncbi, how='inner', on='GCA') # merge CRISPRone annotations with NCBI info, for merging with gbffsummary
print(cn.shape)
# gc = pd.merge(cone,gbsum,how='inner',on='GCF')
# print(gc.shape)
ng = pd.merge(ncbi,gbsum,how='inner',on='GCF')
print(ng.shape)
gcn = pd.merge(cn, gbsum, how='inner', on='GCF') #merfe with gbff summary, all annotation,ncbi data traced back to gbff summary and filepaths on infoserv
print(gcn.shape) #should be 10614
gcn.iloc[1]

(11144, 33)
(8269, 42)
(8269, 48)


Cas Proteins (CRISPRone)                                                                [No System]
GCA                                                                       GCA_000013425.1_ASM1342v1
Name                                                  Staphylococcus aureus subsp. aureus NCTC 8325
No. Arrays (CRISPRone)                                                                           -1
No. Cas Proteins (CRISPRone)                                                                     -1
Report_URL                                        http://omics.soic.indiana.edu/CRISPRone/check....
System Types (CRISPRone)                                                                  No System
Assembly anomaly                                                                               None
Assembly level                                                                      Complete Genome
Assembly method                                                                                None


In [95]:
# cas = gcn['Cas Proteins (CRISPRone)'].apply(lambda x:set(x))
# cas
print(len(set(gcn['Genus'])))
gcn['isCRISPR'] = gcn['Cas Proteins (CRISPRone)'].apply(lambda x:False if ('No System' in x) or (len(x) == 0) else True)
nocrisprtaxa = gcn[gcn['isCRISPR'] == False]
mixedgen = set(gcn[gcn['isCRISPR'] == True]['Genus'])
nocgen = set(nocrisprtaxa['Genus'])
exNonCGen = nocgen - nocgen.intersection(mixedgen)
len(exNonCGen)

976


316

In [97]:
above10g = [x for x in exNonCGen if gcn[gcn['Genus'] == x].shape[0] > 10]
print(len(above10g))
above10g

6


['Borrelia',
 'Chlamydia',
 'Ehrlichia',
 'Prochlorococcus',
 'Blattabacterium',
 'Xylella']

In [104]:
noncg = gcn[gcn['Genus'] == 'Xylella']
# noncg
with open('xylella_nocrispr_file_paths.txt','w') as out:
    for taxa in noncg['Filepath']:
        out.write(taxa + '\n')